In [1]:
!git clone https://github.com/yoniLc/ECCT.git
!ls

fatal: destination path 'ECCT' already exists and is not an empty directory.
ECCT	     codey_pylib.py		quick_start_pytorch_images
README.md    model_out			test.txt
__pycache__  quick_start_pytorch.ipynb


# Overview

We will use ECC_Transfomer as a mechanism for decoding. On top of the model though, we wil add an initial "array shaping" layer. I think we can use a ?transformer? there

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
"""
@author: Yoni Choukroun, choukroun.yoni@gmail.com
Error Correction Code Transformer
https://arxiv.org/abs/2203.14966
"""
import numpy as np
import torch
import os

def Read_pc_matrixrix_alist(fileName):
    with open(fileName, 'r') as file:
        lines = file.readlines()
        columnNum, rowNum = np.fromstring(
            lines[0].rstrip('\n'), dtype=int, sep=' ')
        H = np.zeros((rowNum, columnNum)).astype(int)
        for column in range(4, 4 + columnNum):
            nonZeroEntries = np.fromstring(
                lines[column].rstrip('\n'), dtype=int, sep=' ')
            for row in nonZeroEntries:
                if row > 0:
                    H[row - 1, column - 4] = 1
        return H
#############################################
def row_reduce(mat, ncols=None):
    assert mat.ndim == 2
    ncols = mat.shape[1] if ncols is None else ncols
    mat_row_reduced = mat.copy()
    p = 0
    for j in range(ncols):
        idxs = p + np.nonzero(mat_row_reduced[p:,j])[0]
        if idxs.size == 0:
            continue
        mat_row_reduced[[p,idxs[0]],:] = mat_row_reduced[[idxs[0],p],:]
        idxs = np.nonzero(mat_row_reduced[:,j])[0].tolist()
        idxs.remove(p)
        mat_row_reduced[idxs,:] = mat_row_reduced[idxs,:] ^ mat_row_reduced[p,:]
        p += 1
        if p == mat_row_reduced.shape[0]:
            break
    return mat_row_reduced, p

def get_generator(pc_matrix_):
    assert pc_matrix_.ndim == 2
    pc_matrix = pc_matrix_.copy().astype(bool).transpose()
    pc_matrix_I = np.concatenate((pc_matrix, np.eye(pc_matrix.shape[0], dtype=bool)), axis=-1)
    pc_matrix_I, p = row_reduce(pc_matrix_I, ncols=pc_matrix.shape[1])
    return row_reduce(pc_matrix_I[p:,pc_matrix.shape[1]:])[0]

def get_standard_form(pc_matrix_):
    pc_matrix = pc_matrix_.copy().astype(bool)
    next_col = min(pc_matrix.shape)
    for ii in range(min(pc_matrix.shape)):
        while True:
            rows_ones = ii + np.where(pc_matrix[ii:, ii])[0]
            if len(rows_ones) == 0:
                new_shift = np.arange(ii, min(pc_matrix.shape) - 1).tolist()+[min(pc_matrix.shape) - 1,next_col]
                old_shift = np.arange(ii + 1, min(pc_matrix.shape)).tolist()+[next_col, ii]
                pc_matrix[:, new_shift] = pc_matrix[:, old_shift]
                next_col += 1
            else:
                break
        pc_matrix[[ii, rows_ones[0]], :] = pc_matrix[[rows_ones[0], ii], :]
        other_rows = pc_matrix[:, ii].copy()
        other_rows[ii] = False
        pc_matrix[other_rows] = pc_matrix[other_rows] ^ pc_matrix[ii]
    return pc_matrix.astype(int)
#############################################

def sign_to_bin(x):
    return 0.5 * (1 - x)

def bin_to_sign(x):
    return 1 - 2 * x

def EbN0_to_std(EbN0, rate):
    snr =  EbN0 + 10. * np.log10(2 * rate)
    return np.sqrt(1. / (10. ** (snr / 10.)))

def BER(x_pred, x_gt):
    return torch.mean((x_pred != x_gt).float()).item()

def FER(x_pred, x_gt):
    return torch.mean(torch.any(x_pred != x_gt, dim=1).float()).item()

#############################################
def Get_Generator_and_Parity(n, k, standard_form = False):
    path_pc_mat = os.path.join('ECCT', 'Codes_DB', f'{code.code_type}_N{str(n)}_K{str(k)}')
    if code.code_type in ['POLAR', 'BCH']:
        ParityMatrix = np.loadtxt(path_pc_mat+'.txt')
    elif code.code_type in ['CCSDS', 'LDPC', 'MACKAY']:
        ParityMatrix = Read_pc_matrixrix_alist(path_pc_mat+'.alist')
    else:
        raise Exception(f'Wrong code {code.code_type}')
    if standard_form and code.code_type not in ['CCSDS', 'LDPC', 'MACKAY']:
        ParityMatrix = get_standard_form(ParityMatrix).astype(int)
        GeneratorMatrix = np.concatenate([np.mod(-ParityMatrix[:, min(ParityMatrix.shape):].transpose(),2),np.eye(k)],1).astype(int)
    else:
        GeneratorMatrix = get_generator(ParityMatrix)
    assert np.all(np.mod((np.matmul(GeneratorMatrix, ParityMatrix.transpose())), 2) == 0) and np.sum(GeneratorMatrix) > 0
    return GeneratorMatrix.astype(float), ParityMatrix.astype(float)

class Code():
        pass
code = Code()
code.k = 24
code.n = 49
code.code_type = 'LDPC'
G, H = Get_Generator_and_Parity(code.n, code.k)
code.generator_matrix = torch.from_numpy(G).transpose(0, 1).long()
code.pc_matrix = torch.from_numpy(H).long()
# args.code = codecode = Code()
code.pc_matrix.size()

# Goal, how can we have a **few shot** learner??? I.e. how can we feed the code __into___ the transformer
# Maybe we can "encode" parity check matrix?

torch.Size([28, 49])

In [4]:
!ls

ECCT	     codey_pylib.py		quick_start_pytorch_images
README.md    model_out			test.txt
__pycache__  quick_start_pytorch.ipynb


In [5]:
a = torch.cat(2 * [torch.eye(3, 4).unsqueeze(0).unsqueeze(-1)], dim=-1) * torch.cat(3 * [torch.eye(3, 4).unsqueeze(0)]).unsqueeze(-1)
a.size(), a[1], torch.eye(3, 4) * torch.eye(3, 4)

(torch.Size([3, 3, 4, 2]),
 tensor([[[1., 1.],
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [1., 1.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [1., 1.],
          [0., 0.]]]),
 tensor([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]))

## The Modified Model Code

In [6]:
"""
@author: Yoni Choukroun, choukroun.yoni@gmail.com
Error Correction Code Transformer
https://arxiv.org/abs/2203.14966
"""
from torch.nn import LayerNorm
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy
import logging


def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        if N > 1:
            self.norm2 = LayerNorm(layer.size)

    def forward(self, x, mask):
        for idx, layer in enumerate(self.layers, start=1):
            x = layer(x, mask)
            if idx == len(self.layers)//2 and len(self.layers) > 1:
                x = self.norm2(x)
        return self.norm(x)


class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))


class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        nbatches = query.size(0)
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        x, self.attn = self.attention(query, key, value, mask=mask)

        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

    def attention(self, query, key, value, mask=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(d_k)
        if mask is not None:
            scores = scores.masked_fill(mask, -1e9)
        p_attn = F.softmax(scores, dim=-1)
        if self.dropout is not None:
            p_attn = self.dropout(p_attn)
        return torch.matmul(p_attn, value), p_attn


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.gelu(self.w_1(x))))

############################################################


class ECC_Transformer(nn.Module):
    def __init__(self, n, k, h, d_model, N_dec, pc_adj_size, dropout=0):
        super(ECC_Transformer, self).__init__()
        ####
        c = copy.deepcopy
        attn = MultiHeadedAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_model*4, dropout)

        # MODIFIED
        self.src_embed = torch.nn.Parameter(torch.empty(
            (pc_adj_size + n + (n - k), d_model)))
        # END MODIFIED

        self.decoder = Encoder(EncoderLayer(
            d_model, c(attn), c(ff), dropout), N_dec)
        self.oned_final_embed = torch.nn.Sequential(
            *[nn.Linear(d_model, 1)])

        self.out_fc = nn.Linear(n + (n - k) + pc_adj_size, 1)

        self.n = n
        self.k = k
        self.pc_adj_size = pc_adj_size
        # TODO: renable mask
        # logging.info(f'Mask:\n {self.src_mask}')
        ###
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    # Only allow 1 batch size for now b/c I am feeling lazy rn
    def forward(self, p_check_mat, p_check_mat_adj,  magnitude, syndrome):
        # Modified
        emb = torch.cat([magnitude,
                        syndrome, p_check_mat_adj], -1).unsqueeze(-1)
        # emb = torch.cat(emb.size()[0] * [self.src_embed.unsqueeze(-3)]) * emb
        emb = self.src_embed.unsqueeze(0) * emb
        self.get_mask(self.n, self.k, p_check_mat.view(-1,
                      self.n - self.k, self.n).squeeze(0))  # TODO: reintroduce mask
        # Hm... I feel like this ain't gonna work
        # emb = emb[:, -1 * (2 * self.n - self.k):, :]
        emb = self.decoder(emb, self.src_mask)  # TODO: SRC_MASK
        return self.out_fc(self.oned_final_embed(emb).squeeze(-1))

    def loss(self, z_pred, z2, y):
        loss = F.binary_cross_entropy_with_logits(
            z_pred, sign_to_bin(torch.sign(z2)))
        x_pred = sign_to_bin(torch.sign(-z_pred * torch.sign(y)))
        return loss, x_pred

    def get_mask(self, n, k, p_check_mat, no_mask=False):
        p_check_size = n * (n - k)
        if no_mask:
            # self.src_mask = None
            return

        mask_size_col = self.pc_adj_size + n + n - k
        mask_size_row = self.pc_adj_size + n + n - k

        def build_mask():
            # Set the mask to "on" for all the p_checks
            # TODO: WAIT A SECOND: HOW DOES THIS NOW WORK? WE ARE CATTING ON 0 VECTOR HERE.... b/c col-row = 0
            mask = torch.cat([torch.eye(mask_size_row, mask_size_row), torch.ones(mask_size_row, mask_size_col -
                             mask_size_row)], dim=1)
            # mask = torch.eye(mask_size_row, mask_size_row)
            for ii in range(n-k):
                idx = torch.where(p_check_mat[ii] > 0)[0]
                for jj in idx:
                    for kk in idx:
                        if jj != kk:
                            mask[jj,   kk] += 1
                            mask[kk,  jj] += 1
                            mask[n + ii,  jj] += 1
                            mask[jj,  n + ii] += 1
            src_mask = ~ (mask > 0).unsqueeze(0).unsqueeze(0)
            return src_mask
        src_mask = build_mask()
        a = mask_size_row * mask_size_col
        logging.info(
            f'Self-Attention Sparsity Ratio={100 * torch.sum((src_mask).int()) / a:0.2f}%, Self-Attention Complexity Ratio={100 * torch.sum((~src_mask).int())//2 / a:0.2f}%')
        self.register_buffer('src_mask', src_mask.to(device))


In [7]:
# Thanks to https://github.com/christiansiegel/coding-theory-algorithms/blob/master/LinearBlockCode.py
def GtoP(G):
    """Extract the submatrix P from a Generator Matrix in systematic form.
    Args:
        G: Generator Matrix in systematic form
    Returns:
        Submatrix P of G.
    """
    k = G.size(0)
    n = G.size(1)
    P = G[:k, :n - k]
    return P


def HtoP(H):
    """Extract the submatrix P from a Parity Check Matrix in systematic form.
    Args:
        H: Parity Check Matrix in systematic form
    Returns:
        Submatrix P of G.
    """
    n = H.size(1)
    k = n - H.size(0)
    PK = H[:, n - k:n]
    P = PK.transpose(0, 1)
    return P


def HtoG(H):
    """Convert a Parity Check Matrix in systematic form to a Generator Matrix.
    Args:
        H: Parity Check Matrix in systematic form
    Returns:
        Generator Matrix G
    """
    n = H.size(1)
    k = n - H.size(0)
    P = HtoP(H)
    Ik = torch.eye(k)
    G = torch.concat((P, Ik), axis=1)
    return G


n = 100
k = 30
dR = 5


def gen_random_ldpc(n, k, deg_row):
    iden_left = torch.eye(n - k)
    x = torch.zeros(n-k, k)
    x[:, :(deg_row - 1)] = 1
    perm = torch.stack([torch.randperm(k) for _ in range((n - k))])
    x = x.gather(dim=1, index=perm)
    # TODO: above is wrong use first approach
    rand_ldpc_H = torch.concat([iden_left, x], dim=1)
    G = HtoG(rand_ldpc_H)
    return rand_ldpc_H, G

def pc_to_adj(H):
    x = H.nonzero()
    return x[:, 1]

rand_ldpc_H, G = gen_random_ldpc(n, k, dR)
g = (torch.bernoulli(torch.Tensor([0.1]).repeat(1, k)) @ G) % 2
# Yay, it works :)
((rand_ldpc_H @ G.transpose(0, 1)) % 2).sum(), pc_to_adj(rand_ldpc_H)


(tensor(0.),
 tensor([ 0, 74, 82, 86, 90,  1, 73, 79, 93, 98,  2, 80, 83, 92, 97,  3, 71, 72,
         79, 94,  4, 78, 81, 92, 97,  5, 79, 89, 90, 97,  6, 70, 72, 91, 95,  7,
         82, 87, 88, 96,  8, 72, 76, 77, 79,  9, 71, 75, 88, 98, 10, 72, 82, 91,
         97, 11, 73, 88, 92, 99, 12, 70, 81, 94, 99, 13, 72, 79, 90, 96, 14, 71,
         89, 92, 98, 15, 81, 88, 94, 99, 16, 77, 81, 97, 99, 17, 84, 91, 92, 96,
         18, 72, 85, 88, 93, 19, 77, 80, 88, 91, 20, 72, 75, 91, 94, 21, 77, 78,
         86, 88, 22, 82, 86, 91, 98, 23, 82, 85, 91, 99, 24, 77, 90, 91, 95, 25,
         76, 82, 94, 95, 26, 70, 73, 75, 82, 27, 71, 73, 74, 75, 28, 77, 90, 91,
         95, 29, 72, 77, 92, 96, 30, 70, 71, 84, 93, 31, 75, 76, 89, 94, 32, 88,
         90, 95, 99, 33, 75, 77, 91, 95, 34, 72, 76, 90, 94, 35, 83, 86, 91, 92,
         36, 77, 81, 98, 99, 37, 70, 74, 76, 97, 38, 79, 84, 90, 98, 39, 73, 84,
         89, 91, 40, 77, 82, 85, 99, 41, 71, 72, 87, 93, 42, 84, 93, 95, 97, 43,
         79, 86

In [8]:
# This important will remove all the above stuff..
if False:
	from codey_pylib import *

In [9]:
"""
Implementation of "Error Correction Code Transformer" (ECCT)
https://arxiv.org/abs/2203.14966
@author: Yoni Choukroun, choukroun.yoni@gmail.com
"""
from __future__ import print_function
import argparse
import random
import os
from torch.utils.data import DataLoader
from torch.utils import data
from datetime import datetime
import logging
import time
from torch.optim.lr_scheduler import CosineAnnealingLR

##################################################################
##################################################################

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

##################################################################


class ECC_Dataset(data.Dataset):
    def __init__(self, n, k, sigma, len, zero_cw=True, deg_row=5, always_shuffle_code=False):
        self.sigma = sigma
        self.len = len
        self.n = n
        self.k = k
        # self.generator_matrix = code.generator_matrix.transpose(0, 1)
        # self.pc_matrix = code.pc_matrix.transpose(0, 1)
        self.deg_row = deg_row
        self.zero_word = torch.zeros((self.k)).long() if zero_cw else None
        self.zero_cw = torch.zeros((self.n)).long() if zero_cw else None
        self.always_shuffle_code = always_shuffle_code
        self.shuffle_rand_code()
    
    def shuffle_rand_code(self):
        pc_matrix, generator_matrix = gen_random_ldpc(self.n, self.k, self.deg_row)
        self.pc_matrix = pc_matrix
        self.generator_matrix = generator_matrix

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        if self.always_shuffle_code:
            self.shuffle_rand_code()
        if self.zero_cw is None:
            m = torch.randint(0, 2, (1, self.k)).squeeze().unsqueeze(-2)
            x = torch.matmul(m.float(), self.generator_matrix).transpose(-1, -2) % 2
            x = x.squeeze(-1)
        else:
            m = self.zero_word
            x = self.zero_cw
        z = torch.randn(self.n) * random.choice(self.sigma)
        y = bin_to_sign(x) + z
        magnitude = torch.abs(y)
        syndrome = torch.matmul(self.pc_matrix.long(), sign_to_bin(torch.sign(y)).long().unsqueeze(-1)) % 2
        syndrome = bin_to_sign(syndrome).squeeze(-1)
        return self.pc_matrix.flatten().float(), pc_to_adj(self.pc_matrix), m.float(), x.float(), z.float(), y.float(), magnitude.float(), syndrome.float()

dataset = ECC_Dataset(n, k, [1], 1)
a = dataset.__getitem__(1)
a[0].size(),  a[-2].size(), a[-1].size()


(torch.Size([7000]), torch.Size([100]), torch.Size([70]))

In [10]:
h = 1
N_dec = 1 # CHanged from 6
d_model = 20 # default is 32 but we are adding parity check info...
pc_adj_size = pc_to_adj( gen_random_ldpc(n, k, deg_row=5)[0]).size()[-1]
test_model = ECC_Transformer(n, k, h, d_model, N_dec, pc_adj_size, dropout=0).to(device)
r = None
with torch.no_grad():
	r = test_model(a[0].to(device), a[1].to(device), a[6].to(device), a[7].to(device))
r.heaviside(torch.Tensor([0]).to(device))

/tmp/ipykernel_32/1223232003.py:182: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  f'Self-Attention Sparsity Ratio={100 * torch.sum((src_mask).int()) / a:0.2f}%, Self-Attention Complexity Ratio={100 * torch.sum((~src_mask).int())//2 / a:0.2f}%')


tensor([[1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
         1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
         1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
         1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 1., 0., 0., 0., 1., 0.]], device='cuda:0')

In [11]:
torch.tensor([[2, 1], [3, 2]])[:][1]

tensor([3, 2])

In [14]:
from IPython.display import display, clear_output
##################################################################
##################################################################
# logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)
# logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)


def train(model, device, train_loader, optimizer, epoch, LR, shuffle_code_round):
    model.train()
    cum_loss = cum_ber = cum_fer = cum_samples = 0
    t = time.time()
    for batch_idx, (parity_check, pc_adj, m, x, z, y, magnitude, syndrome) in enumerate(
            train_loader):
        if batch_idx % shuffle_code_round == 0:
            train_loader.dataset.shuffle_rand_code()
        z_mul = (y * bin_to_sign(x))
        z_pred = model(parity_check.to(device), pc_adj.to(device),
                       magnitude.to(device), syndrome.to(device))
        loss, x_pred = model.loss(-z_pred, z_mul.to(device), y.to(device))
        model.zero_grad()
        loss.backward()
        optimizer.step()
        ###
        ber = BER(x_pred, x.to(device))
        fer = FER(x_pred, x.to(device))

        cum_loss += loss.item() * x.shape[0]
        cum_ber += ber * x.shape[0]
        cum_fer += fer * x.shape[0]
        cum_samples += x.shape[0]
        if (batch_idx+1) % 500 == 0 or batch_idx == len(train_loader) - 1:
            logging.info(
                f'Training epoch {epoch}, Batch {batch_idx + 1}/{len(train_loader)}: LR={LR:.2e}, Loss={cum_loss / cum_samples:.2e} BER={cum_ber / cum_samples:.2e} FER={cum_fer / cum_samples:.2e}')
    logging.info(f'Epoch {epoch} Train Time {time.time() - t}s\n')
    return cum_loss / cum_samples, cum_ber / cum_samples, cum_fer / cum_samples


##################################################################

def test(model, device, test_loader_list, EbNo_range_test, min_FER=10):
    model.eval()
    test_loss_list, test_loss_ber_list, test_loss_fer_list, cum_samples_all = [], [], [], []
    t = time.time()
    with torch.no_grad():
        for ii, test_loader in enumerate(test_loader_list):
            test_loss = test_ber = test_fer = cum_count = 0.
            while True:
                (parity_check, pc_adj, jm, x, z, y, magnitude,
                 syndrome) = next(iter(test_loader))
                z_mul = (y * bin_to_sign(x))
                z_pred = model(parity_check.to(device), pc_adj.to(device),
                               magnitude.to(device), syndrome.to(device))
                loss, x_pred = model.loss(-z_pred,
                                          z_mul.to(device), y.to(device))

                test_loss += loss.item() * x.shape[0]

                test_ber += BER(x_pred, x.to(device)) * x.shape[0]
                test_fer += FER(x_pred, x.to(device)) * x.shape[0]
                cum_count += x.shape[0]
                # cum count before 1e5
                if (min_FER > 0 and test_fer > min_FER and cum_count > 10) or cum_count >= 1e9:
                    if cum_count >= 1e9:
                        print(
                            f'Number of samples threshold reached for EbN0:{EbNo_range_test[ii]}')
                    else:
                        print(
                            f'FER count threshold reached for EbN0:{EbNo_range_test[ii]}')
                    break
            cum_samples_all.append(cum_count)
            test_loss_list.append(test_loss / cum_count)
            test_loss_ber_list.append(test_ber / cum_count)
            test_loss_fer_list.append(test_fer / cum_count)
            print(
                f'Test EbN0={EbNo_range_test[ii]}, BER={test_loss_ber_list[-1]:.2e}')
        ###
        logging.info('\nTest Loss ' + ' '.join(
            ['{}: {:.2e}'.format(ebno, elem) for (elem, ebno)
             in
             (zip(test_loss_list, EbNo_range_test))]))
        logging.info('Test FER ' + ' '.join(
            ['{}: {:.2e}'.format(ebno, elem) for (elem, ebno)
             in
             (zip(test_loss_fer_list, EbNo_range_test))]))
        logging.info('Test BER ' + ' '.join(
            ['{}: {:.2e}'.format(ebno, elem) for (elem, ebno)
             in
             (zip(test_loss_ber_list, EbNo_range_test))]))
        logging.info('Test -ln(BER) ' + ' '.join(
            ['{}: {:.2e}'.format(ebno, -np.log(elem)) for (elem, ebno)
             in
             (zip(test_loss_ber_list, EbNo_range_test))]))
    logging.info(
        f'# of testing samples: {cum_samples_all}\n Test Time {time.time() - t} s\n')
    return test_loss_list, test_loss_ber_list, test_loss_fer_list

##################################################################
##################################################################
##################################################################


model = None


def main(n, k, deg_row, save_path,):
    global model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #################################
    h = 4  # changed from 8...
    lr = 1e-4
    epochs = 1000
    batch_size = 1
    # Use a new random code after 32 runs, we do not want this to be too high as we are
    # trying to learn a **general** decoder
    rand_code_shuffle_len = 8
    workers = 1
    N_dec = 3  # CHanged from 6
    d_model = 40  # default is 32 but we are adding parity check info...
    adj_size = (n - k) * deg_row
    model = ECC_Transformer(n, k, h, d_model, N_dec,
                            adj_size,  dropout=0).to(device)
    model = torch.load(os.path.join(save_path, 'best_model'))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)

    # We want a train size of about 400 * batch_size
    train_size = batch_size * \
        math.floor(400 / rand_code_shuffle_len) * rand_code_shuffle_len

    logging.info(model)
    logging.info(
        f'# of Parameters: {np.sum([np.prod(p.shape) for p in model.parameters()])}')
    #################################
    EbNo_range_test = range(4, 7)
    EbNo_range_train = range(2, 8)
    test_batch_size = 1
    std_train = [EbN0_to_std(ii, k / n) for ii in EbNo_range_train]
    std_test = [EbN0_to_std(ii, k / n) for ii in EbNo_range_test]
    train_dataloader = DataLoader(ECC_Dataset(n, k, std_train, len=train_size, deg_row=deg_row, zero_cw=True, always_shuffle_code=False), batch_size=int(batch_size),
                                  shuffle=True, num_workers=workers)
    test_dataloader_list = [DataLoader(ECC_Dataset(n, k, [std_test[ii]], deg_row=deg_row, len=int(test_batch_size), zero_cw=False, always_shuffle_code=True),
                                       batch_size=int(test_batch_size), shuffle=False, num_workers=workers) for ii in range(len(std_test))]
    #################################
    # TODO: increase the batch size so loss is a better metric for saving
    best_loss = float('inf')
    last_save_epoch = -1
    for epoch in range(1, epochs + 1):
        loss, ber, fer = train(model, device, train_dataloader, optimizer,
                               epoch, LR=scheduler.get_last_lr()[0], shuffle_code_round=rand_code_shuffle_len)
        scheduler.step()
        if loss < best_loss and epoch - last_save_epoch > 15:
            best_loss = loss
            last_save_epoch = epoch
            torch.save(model, os.path.join(save_path, 'best_model'))
            print("Saving Model at Eopch", epoch)
        if epoch % 300 == 0 or epoch in [1, epochs]:
            test_loss_list, test_loss_ber_list, test_loss_fer_list = test(
                model, device, test_dataloader_list, EbNo_range_test)
            print("Losses for test of", test_loss_list,
                  test_loss_ber_list, test_loss_fer_list)
        # clear_output(wait=True)
        print(f"Epoch {epoch} finished, loss: {loss}, ber: {ber}, fer: {fer}")


GLOBAL_N = 50
GLOBAL_K = 15
GLOBAL_DEG_ROW = 5
main(GLOBAL_N, GLOBAL_K, GLOBAL_DEG_ROW, 'model_out')


/tmp/ipykernel_32/1223232003.py:182: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  f'Self-Attention Sparsity Ratio={100 * torch.sum((src_mask).int()) / a:0.2f}%, Self-Attention Complexity Ratio={100 * torch.sum((~src_mask).int())//2 / a:0.2f}%')


FER count threshold reached for EbN0:4
Test EbN0=4, BER=1.13e-01
FER count threshold reached for EbN0:5
Test EbN0=5, BER=9.50e-02
FER count threshold reached for EbN0:6
Test EbN0=6, BER=4.46e-02
Losses for test of [0.3594899096272208, 0.30780491046607494, 0.19489356474234507] [0.11272727020762184, 0.09499999756614368, 0.04461538333159227] [1.0, 0.9166666666666666, 0.8461538461538461]
Epoch 1 finished, loss: 0.3049606670066714, ber: 0.09449999757111073, fer: 0.97
Epoch 2 finished, loss: 0.3273327129147947, ber: 0.10349999709054827, fer: 0.965
Epoch 3 finished, loss: 0.2972557982429862, ber: 0.0924999972525984, fer: 0.9575
Epoch 4 finished, loss: 0.3024458884168416, ber: 0.09479999737814068, fer: 0.96
Epoch 5 finished, loss: 0.3161900403071195, ber: 0.10189999711699783, fer: 0.97
Epoch 6 finished, loss: 0.31235774494707585, ber: 0.10054999725893139, fer: 0.9825
Epoch 7 finished, loss: 0.3207790886517614, ber: 0.10514999704435468, fer: 0.97
Epoch 8 finished, loss: 0.31178453031461684, ber

In [ ]:
if False:
	torch.save(model, os.path.join('model_out', 'best_model'))

In [ ]:
!cat ECCT/Main.py

## Fine tuning test

Fine tune the transformer to work on only **1** specific ECC

In [ ]:

class ECC_DatasetFineTune(data.Dataset):
    def __init__(self, n, k, sigma, len, parity_check_mat, gen_mat, zero_cw=True):
        self.sigma = sigma
        self.len = len
        self.n = n
        self.k = k
        # self.generator_matrix = code.generator_matrix.transpose(0, 1)
        # self.pc_matrix = code.pc_matrix.transpose(0, 1)
        self.zero_word = torch.zeros((self.k)).long() if zero_cw else None
        self.zero_cw = torch.zeros((self.n)).long() if zero_cw else None
        self.pc_mat = parity_check_mat
        self.gen_mat = gen_mat

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        deg_row_avg = 5  # TODO: vary me
        pc_matrix, generator_matrix = self.pc_mat, self.gen_mat
        if self.zero_cw is None:
            m = torch.randint(0, 2, (1, self.k)).squeeze().unsqueeze(-2)
            x = torch.matmul(m.float(), generator_matrix).transpose(-1, -2) % 2
            x = x.squeeze(-1)
        else:
            m = self.zero_word
            x = self.zero_cw
        z = torch.randn(self.n) * random.choice(self.sigma)
        y = bin_to_sign(x) + z
        magnitude = torch.abs(y)
        syndrome = torch.matmul(pc_matrix.long(), sign_to_bin(
            torch.sign(y)).long().unsqueeze(-1)) % 2
        syndrome = bin_to_sign(syndrome).squeeze(-1)
        return pc_matrix.flatten().float(), m.float(), x.float(), z.float(), y.float(), magnitude.float(), syndrome.float()


n = 40
k = 5
deg_row_avg = 5
pc_matrix, generator_matrix = gen_random_ldpc(n, k, deg_row_avg)
dataset = ECC_DatasetFineTune(n, k, [1], 1, pc_matrix, generator_matrix)
a = dataset.__getitem__(1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#################################
h = 4  # changed from 8...
lr = 1e-4
epochs = 1000
batch_size = 1
workers = 1
N_dec = 3  # CHanged from 6
d_model = 40  # default is 32 but we are adding parity check info...
model = ECC_Transformer(n, k, h, d_model, N_dec, dropout=0).to(device)
save_path = 'model_out'
model = torch.load(os.path.join(save_path, 'best_model'))

In [ ]:

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
logging.info(model)
logging.info(
    f'# of Parameters: {np.sum([np.prod(p.shape) for p in model.parameters()])}')
#################################
EbNo_range_test = range(4, 7)
EbNo_range_train = range(2, 8)
test_batch_size = 1
std_train = [EbN0_to_std(ii, k / n) for ii in EbNo_range_train]
std_test = [EbN0_to_std(ii, k / n) for ii in EbNo_range_test]
train_dataloader = DataLoader(ECC_DatasetFineTune(n, k, std_train, batch_size * 20, pc_matrix, generator_matrix, zero_cw=True), batch_size=int(batch_size),
                              shuffle=True, num_workers=workers)
test_dataloader_list = [DataLoader(ECC_DatasetFineTune(n, k, [std_test[ii]], int(test_batch_size), pc_matrix, generator_matrix,  zero_cw=False),
                                   batch_size=int(test_batch_size), shuffle=False, num_workers=workers) for ii in range(len(std_test))]
#################################
best_loss = float('inf')
last_save_epoch = -1
for epoch in range(1, epochs + 1):
    loss, ber, fer = train(model, device, train_dataloader, optimizer,
                           epoch, LR=scheduler.get_last_lr()[0])
    scheduler.step()
    if loss < best_loss and epoch - last_save_epoch > 30:
        best_loss = loss
        last_save_epoch = epoch
        torch.save(model, os.path.join(save_path, 'best_model_fine_tuned'))
    if epoch % 300 == 0 or epoch in [1, epochs]:
        test_loss_list, test_loss_ber_list, test_loss_fer_list = test(
            model, device, test_dataloader_list, EbNo_range_test)
        print("Losses for test of", test_loss_list,
              test_loss_ber_list, test_loss_fer_list)
    # clear_output(wait=True)
    print(f"Epoch {epoch} finished", loss, ber, fer)
